# 학술 논문 작성을 위한 Multi-Agent 시스템

## 1. 소개

### 1.1. 개요

이 노트북은 **심리학 논문 작성을 돕는 7가지 전문 에이전트**를 구현합니다:

1. **Literature Search Agent** 📚: PubMed에서 관련 논문 검색 및 CSV 저장
2. **Introduction Writer Agent** ✍️: 초록을 기반으로 서론 작성
3. **APA 7 Citation Checker Agent** 📝: APA 7판 형식 검증
4. **Coherence Checker Agent** 🔗: 문맥과 논리적 흐름 검증
5. **Summarizing Agent** 📋: 작성된 내용 요약
6. **Figure Generation Agent** 📊: 연구 그림 생성
7. **Reader Accessibility Agent** 👥: 가독성 평가 (대학 신입생 수준)

### 🎯 1.2 학습 목표

- 논문 작성 과정을 자동화하는 multi-agent 시스템 구축
- 각 에이전트가 특정 역할을 수행하도록 설계
- 실제 학술 논문 작성에 활용 가능한 도구 제작

## 2. 환경 설정

In [28]:
import json
import os
import re
import requests
from bs4 import BeautifulSoup
from typing import List, Dict
from dotenv import load_dotenv
import google.generativeai as genai
from google.generativeai.types import FunctionDeclaration, Tool
from google.generativeai import protos
import inspect
import PyPDF2
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 환경 변수 로드
load_dotenv()

# API 설정
API_PROVIDER = "gemini"
MODEL = "gemini-2.5-flash"

print(f"✅ 사용할 모델: {MODEL} (Provider: {API_PROVIDER})")

# Gemini API 초기화
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
print("Using Google Generative AI for Gemini")

✅ 사용할 모델: gemini-2.5-flash (Provider: gemini)
Using Google Generative AI for Gemini


In [29]:
# 통합 API 래퍼 클래스
class UnifiedLLMClient:
    """Gemini API를 사용하는 래퍼 클래스"""

    def __init__(self, provider, model):
        self.provider = provider
        self.model = model

    def _convert_tools_to_gemini(self, tools: List) -> List[Tool]:
        """Python 함수를 Gemini Tool 형식으로 변환"""
        function_declarations = []

        for tool in tools:
            description = tool.__doc__ or f"Execute {tool.__name__}"
            description = description.strip()

            sig = inspect.signature(tool)
            parameters = {"type": "object", "properties": {}, "required": []}

            for param_name, param in sig.parameters.items():
                param_type = "string"
                if param.annotation != inspect.Parameter.empty:
                    if param.annotation == int:
                        param_type = "integer"
                    elif param.annotation == float:
                        param_type = "number"
                    elif param.annotation == bool:
                        param_type = "boolean"

                parameters["properties"][param_name] = {"type": param_type}

                if param.default == inspect.Parameter.empty:
                    parameters["required"].append(param_name)

            func_decl = FunctionDeclaration(
                name=tool.__name__,
                description=description,
                parameters=parameters
            )
            function_declarations.append(func_decl)

        return [Tool(function_declarations=function_declarations)]

    def chat_completions_create(self, messages, tools, max_turns=5):
        """통합 chat completion 인터페이스"""
        gemini_model = genai.GenerativeModel(
            model_name=self.model,
            tools=self._convert_tools_to_gemini(tools)
        )

        prompt = messages[-1]["content"] if isinstance(messages, list) else str(messages)
        tool_map = {tool.__name__: tool for tool in tools}

        chat = gemini_model.start_chat()
        response = chat.send_message(prompt)

        # 함수 호출 루프
        for turn in range(max_turns):
            function_called = False

            for part in response.parts:
                if part.function_call:
                    function_name = part.function_call.name
                    function_args = dict(part.function_call.args)

                    function_result = tool_map[function_name](**function_args)

                    function_response_part = protos.Part(
                        function_response=protos.FunctionResponse(
                            name=function_name,
                            response={"result": str(function_result)}
                        )
                    )

                    response = chat.send_message(function_response_part)
                    function_called = True
                    break

            if not function_called:
                break

        class MockChoice:
            def __init__(self, text):
                self.message = type('obj', (object,), {'content': text})()

        class MockResponse:
            def __init__(self, text):
                self.choices = [MockChoice(text)]

        return MockResponse(response.text)

# 클라이언트 초기화
client = UnifiedLLMClient(API_PROVIDER, MODEL)
print("✅ LLM 클라이언트 초기화 완료")

✅ LLM 클라이언트 초기화 완료


## 3. Agent 1: Literature Search Agent 📚

**PubMed와 arXiv**에서 논문을 검색하고 CSV 파일로 저장하는 에이전트입니다.

### 📚 PubMed 검색
- **데이터 소스**: PubMed API (NCBI E-utilities via Biopython)
- **기술 스택**: Biopython의 `Bio.Entrez` 모듈 사용
- **특징**:
  - 의학, 생명과학, 심리학 논문
  - MeSH 키워드 자동 추출
  - DOI 및 PMID 포함
  - 🚫 제외 단어 (기본값):
    - `psychosis` - 정신병 관련 논문 제외
    - `neuro*` - neuroscience, neurology, neuropsychology 등 neuro로 시작하는 모든 단어 제외

### 📄 arXiv 검색
- **데이터 소스**: arXiv API
- **카테고리**: cs.CL (Computation and Language), q-bio.NC (Neurons and Cognition)
- **특징**:
  - 컴퓨터과학 + 인지과학 분야 논문
  - 최신 프리프린트 논문 포함
  - 카테고리 기반 필터링

### 📊 출력 형식
CSV 파일: `Index, Source, ID, Title, Authors, Year, Journal, Abstract, DOI, Keywords, Link`

### 🎯 사용 예시
```python
# 기본 사용 (PubMed + arXiv 모두 검색)
literature_search_agent(
    topic="emotion recognition",
    keywords="large language model",
    max_results=10,
    email="your.email@example.com"
)

# PubMed만 검색 (제외 단어 커스터마이즈)
literature_search_agent(
    topic="cognitive training",
    keywords="working memory aging",
    sources=['pubmed'],
    exclude_terms=['psychosis', 'neuro*', 'schizophrenia']
)

# arXiv만 검색 (다른 카테고리)
literature_search_agent(
    topic="sentiment analysis",
    keywords="deep learning",
    sources=['arxiv'],
    arxiv_categories=['cs.CL', 'cs.AI']
)
```

### 📝 참고사항
- PubMed 검색 시 NCBI 정책에 따라 이메일 주소가 필요합니다
- `neuro*`는 PubMed에서 neuro로 시작하는 모든 단어를 제외하는 와일드카드입니다
- arXiv는 소속 정보를 제공하지 않으므로 카테고리 기반으로만 필터링됩니다
- 두 데이터베이스 모두 별도의 CSV 파일로 저장됩니다

In [68]:
from Bio import Entrez
import csv
from datetime import datetime
import time
import urllib.request
import urllib.parse
import xml.etree.ElementTree as ET

def search_pubmed(topic: str, keywords: str, max_results: int = 10, email: str = "your.email@example.com", exclude_terms: list = None) -> str:
    """
    Search for academic papers using PubMed API (via Biopython's Entrez) and save to CSV file.
    
    This function uses the official NCBI E-utilities API through Biopython.
    Google Scholar API is NOT used.
    
    Args:
        topic: Research topic or title
        keywords: Psychological constructs or key terms to search
        max_results: Maximum number of results to return (default: 10)
        email: Email for NCBI API (required by NCBI policy)
        exclude_terms: List of terms to exclude from search (default: ['psychosis', 'neuro*'])
    
    Returns:
        Message about the saved CSV file with search results
    """
    # NCBI requires email for API access
    Entrez.email = email
    
    # 기본 제외 단어 설정 (neuro*는 neuro로 시작하는 모든 단어 제외)
    if exclude_terms is None:
        exclude_terms = ['psychosis', 'neuro*','Neuro*', 'EEG']
    
    try:
        # 검색 쿼리 생성 - PubMed는 AND로 단어를 자동 연결
        base_query = f"{topic} {keywords}"
        
        # 제외 단어가 있으면 NOT 연산자로 추가
        if exclude_terms:
            exclude_parts = ' NOT '.join(exclude_terms)
            search_query = f"({base_query}) NOT ({exclude_parts})"
        else:
            search_query = base_query
        
        print(f"🔍 PubMed 검색 중: {search_query}")
        
        # 1단계: PubMed ID 검색
        search_handle = Entrez.esearch(
            db="pubmed",
            term=search_query,
            retmax=max_results,
            sort="relevance"
        )
        search_results = Entrez.read(search_handle)
        search_handle.close()
        
        id_list = search_results.get("IdList", [])
        
        if not id_list:
            raise Exception(f"No papers found in PubMed for query: {search_query}")
        
        print(f"✅ 검색된 논문 수: {len(id_list)}")
        if exclude_terms:
            print(f"🚫 제외된 단어: {', '.join(exclude_terms)}")
        
        # 2단계: 논문 상세 정보 가져오기
        papers = []
        batch_size = 10  # 한 번에 가져올 논문 수
        
        for i in range(0, len(id_list), batch_size):
            batch_ids = id_list[i:i+batch_size]
            
            print(f"📥 논문 정보 수집 중: {i+1}-{min(i+batch_size, len(id_list))}/{len(id_list)}")
            
            # efetch로 상세 정보 가져오기
            fetch_handle = Entrez.efetch(
                db="pubmed",
                id=batch_ids,
                rettype="medline",
                retmode="xml"
            )
            records = Entrez.read(fetch_handle)
            fetch_handle.close()
            
            # 각 논문 정보 추출
            for record in records.get('PubmedArticle', []):
                paper_info = _extract_pubmed_info(record)
                if paper_info:
                    papers.append(paper_info)
            
            time.sleep(0.5)  # API 호출 제한 준수
        
        if not papers:
            raise Exception("Could not extract paper information")
        
        # CSV 파일로 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"literature_search_pubmed_{timestamp}.csv"
        
        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            fieldnames = ['Index', 'Source', 'ID', 'Title', 'Authors', 'Year', 'Journal', 'Abstract', 'DOI', 'Keywords', 'Link']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for idx, paper in enumerate(papers, 1):
                paper['Index'] = idx
                paper['Source'] = 'PubMed'
                writer.writerow(paper)
        
        print(f"💾 CSV 파일 저장 완료: {filename}")
        
        exclude_msg = f", excluded: {', '.join(exclude_terms)}" if exclude_terms else ""
        return f"Successfully found and saved {len(papers)} papers to '{filename}'{exclude_msg}"
        
    except Exception as e:
        print(f"❌ PubMed 검색 중 오류 발생: {str(e)}")
        return f"Error: {str(e)}"


def search_arxiv(
    topic: str,
    keywords: str,
    max_results: int = 10,
    categories: list = None
) -> str:
    """
    Search for academic papers using arXiv API and save to CSV file.

    Args:
        topic: Research topic or title
        keywords: Key terms to search
        max_results: Maximum number of results to fetch (default: 10)
        categories: List of arXiv categories (default: ['cs.CL', 'q-bio.NC'])

    Returns:
        Message about the saved CSV file with search results
    """
    # 기본 카테고리 설정
    if categories is None:
        categories = ['cs.CL', 'q-bio.NC']

    try:
        # 검색 쿼리 생성
        search_terms = f"{topic} {keywords}"

        # 카테고리 필터 추가
        category_query = ' OR '.join([f'cat:{cat}' for cat in categories])
        full_query = f"({category_query}) AND ({search_terms})"

        print(f"🔍 arXiv 검색 중: {full_query}")
        print(f"📂 카테고리: {', '.join(categories)}")

        # arXiv API 호출
        base_url = 'http://export.arxiv.org/api/query?'
        query_params = {
            'search_query': full_query,
            'start': 0,
            'max_results': max_results,
            'sortBy': 'relevance',
            'sortOrder': 'descending'
        }

        url = base_url + urllib.parse.urlencode(query_params)

        with urllib.request.urlopen(url) as response:
            xml_data = response.read()

        # XML 파싱
        root = ET.fromstring(xml_data)

        # Namespace 처리
        ns = {
            'atom': 'http://www.w3.org/2005/Atom',
            'arxiv': 'http://arxiv.org/schemas/atom'
        }

        entries = root.findall('atom:entry', ns)

        if not entries:
            raise Exception("No papers found in arXiv")

        print(f"✅ 검색된 논문 수: {len(entries)}")

        # 논문 정보 추출
        papers = []

        for entry in entries:
            paper_info = _extract_arxiv_info(entry, ns)
            if paper_info:
                papers.append(paper_info)

        if not papers:
            raise Exception("Could not extract paper information")

        # CSV 파일로 저장
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"literature_search_arxiv_{timestamp}.csv"

        with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
            fieldnames = ['Index', 'Source', 'ID', 'Title', 'Authors', 'Year', 'Journal', 'Abstract', 'DOI', 'Keywords', 'Link']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for idx, paper in enumerate(papers, 1):
                paper['Index'] = idx
                paper['Source'] = 'arXiv'
                writer.writerow(paper)

        print(f"💾 CSV 파일 저장 완료: {filename}")

        return f"Successfully found and saved {len(papers)} papers to '{filename}'"

    except Exception as e:
        print(f"❌ arXiv 검색 중 오류 발생: {str(e)}")
        return f"Error: {str(e)}"


def _extract_pubmed_info(record):
    """PubMed 레코드에서 필요한 정보 추출"""
    try:
        medline_citation = record.get('MedlineCitation', {})
        article = medline_citation.get('Article', {})
        
        # PMID
        pmid = str(medline_citation.get('PMID', ''))
        
        # 제목
        title = str(article.get('ArticleTitle', 'No title'))
        
        # 저자 정보
        authors = []
        author_list = article.get('AuthorList', [])
        for author in author_list[:3]:  # 최대 3명
            if 'LastName' in author and 'Initials' in author:
                authors.append(f"{author['LastName']} {author['Initials']}")
        
        author_string = ', '.join(authors) if authors else 'No authors'
        if len(author_list) > 3:
            author_string += ' et al.'
        
        # 출판 연도
        pub_date = article.get('Journal', {}).get('JournalIssue', {}).get('PubDate', {})
        year = pub_date.get('Year', 'N/A')
        
        # 저널
        journal = str(article.get('Journal', {}).get('Title', 'Unknown journal'))
        
        # 초록
        abstract = ""
        if 'Abstract' in article:
            abstract_parts = article['Abstract'].get('AbstractText', [])
            if abstract_parts:
                abstract = ' '.join([str(part) for part in abstract_parts])
        
        # DOI
        doi = ""
        elocation_list = article.get('ELocationID', [])
        for eloc in elocation_list:
            if hasattr(eloc, 'attributes') and eloc.attributes.get('EIdType') == 'doi':
                doi = str(eloc)
                break
        
        # 키워드 (MeSH terms)
        keywords = []
        mesh_list = medline_citation.get('MeshHeadingList', [])
        for mesh in mesh_list[:5]:  # 최대 5개
            descriptor = mesh.get('DescriptorName', '')
            if descriptor:
                keywords.append(str(descriptor))
        
        keyword_list = medline_citation.get('KeywordList', [])
        for kw_group in keyword_list:
            for kw in kw_group[:5]:  # 최대 5개
                keywords.append(str(kw))
        
        keywords_string = '; '.join(keywords[:10]) if keywords else 'No keywords'
        
        # PubMed 링크
        link = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/" if pmid else 'No link'
        
        return {
            'ID': pmid,
            'Title': title,
            'Authors': author_string,
            'Year': year,
            'Journal': journal,
            'Abstract': abstract[:500] + '...' if len(abstract) > 500 else abstract,
            'DOI': doi,
            'Keywords': keywords_string,
            'Link': link
        }
        
    except Exception as e:
        print(f"⚠️ 논문 정보 추출 중 오류: {str(e)}")
        return None


def _extract_arxiv_info(entry, ns):
    """arXiv 엔트리에서 필요한 정보 추출"""
    try:
        # arXiv ID
        arxiv_id = entry.find('atom:id', ns).text.split('/abs/')[-1]
        
        # 제목
        title = entry.find('atom:title', ns).text.strip().replace('\n', ' ')
        
        # 저자 정보
        authors = []
        
        for author in entry.findall('atom:author', ns):
            name = author.find('atom:name', ns).text
            authors.append(name)
        
        # 저자 문자열 생성 (최대 3명)
        author_string = ', '.join(authors[:3]) if authors else 'No authors'
        if len(authors) > 3:
            author_string += ' et al.'
        
        # 출판일
        published = entry.find('atom:published', ns).text
        year = published[:4] if published else 'N/A'
        
        # 카테고리 (저널 대신)
        categories = []
        for cat in entry.findall('atom:category', ns):
            categories.append(cat.get('term'))
        journal = 'arXiv: ' + ', '.join(categories[:3]) if categories else 'arXiv'
        
        # 초록
        summary = entry.find('atom:summary', ns).text.strip().replace('\n', ' ')
        abstract = summary[:500] + '...' if len(summary) > 500 else summary
        
        # DOI (있는 경우)
        doi_elem = entry.find('arxiv:doi', ns)
        doi = doi_elem.text if doi_elem is not None else ''
        
        # 키워드 (카테고리로 대체)
        keywords = '; '.join(categories)
        
        # arXiv 링크
        link = f"https://arxiv.org/abs/{arxiv_id}"
        
        return {
            'ID': arxiv_id,
            'Title': title,
            'Authors': author_string,
            'Year': year,
            'Journal': journal,
            'Abstract': abstract,
            'DOI': doi,
            'Keywords': keywords,
            'Link': link
        }
        
    except Exception as e:
        print(f"⚠️ 논문 정보 추출 중 오류: {str(e)}")
        return None


print("✅ Literature Search Tools 정의 완료 (PubMed + arXiv)")

✅ Literature Search Tools 정의 완료 (PubMed + arXiv)


In [69]:
def literature_search_agent(
    topic: str, 
    keywords: str, 
    max_results: int = 50, 
    email: str = "your.email@example.com",
    sources: list = None,
    exclude_terms: list = None,
    arxiv_categories: list = None
) -> dict:
    """
    Search for academic papers from multiple sources and save to CSV.
    
    This agent searches both PubMed and arXiv databases.
    - PubMed: Uses Biopython's Entrez module
    - arXiv: Searches specified categories
    
    Args:
        topic: Research topic or title
        keywords: Key terms to search
        max_results: Maximum number of results per source (default: 10)
        email: Email for NCBI API (required by NCBI policy)
        sources: List of sources to search (default: ['pubmed', 'arxiv'])
        exclude_terms: List of terms to exclude from PubMed search (default: ['psychosis'])
        arxiv_categories: List of arXiv categories (default: ['cs.CL', 'q-bio.NC'])
    
    Returns:
        Dictionary containing agent name and result messages
    """
    # 기본 소스 설정
    if sources is None:
        sources = ['pubmed', 'arxiv']
    
    results = []
    
    # PubMed 검색
    if 'pubmed' in sources:
        print("\n" + "="*60)
        print("📚 PubMed 검색 시작")
        print("="*60)
        pubmed_result = search_pubmed(topic, keywords, max_results, email, exclude_terms)
        results.append(f"[PubMed] {pubmed_result}")
    
    # arXiv 검색
    if 'arxiv' in sources:
        print("\n" + "="*60)
        print("📄 arXiv 검색 시작")
        print("="*60)
        arxiv_result = search_arxiv(topic, keywords, max_results, arxiv_categories)
        results.append(f"[arXiv] {arxiv_result}")
    
    combined_result = "\n\n".join(results)
    
    return {
        "agent": "Literature Search Agent (PubMed + arXiv)",
        "result": combined_result
    }

print("✅ Literature Search Agent 정의 완료 (PubMed + arXiv)")

✅ Literature Search Agent 정의 완료 (PubMed + arXiv)


### 테스트: Literature Search Agent

In [70]:
# 예시 실행
print("📚 Literature Search Agent 테스트\n")
print("="*60)

# 이메일 설정 (실제 사용 시 본인의 이메일로 변경)
user_email = "student@snu.ac.kr"  # NCBI 정책상 필수

result = literature_search_agent(
    topic="emotion recognition",
    keywords="large language model cognitive appraisal empathy",  # 쉼표 제거
    max_results=50,
    email=user_email,
    sources=['pubmed', 'arxiv'],  # 두 소스 모두 검색
    arxiv_categories=['cs.CL', 'q-bio.NC']  # cs.CL과 cognitive science 카테고리
)

print(f"\n{'='*60}")
print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

📚 Literature Search Agent 테스트


📚 PubMed 검색 시작
🔍 PubMed 검색 중: (emotion recognition large language model cognitive appraisal empathy) NOT (psychosis NOT neuro* NOT Neuro* NOT EEG)
❌ PubMed 검색 중 오류 발생: No papers found in PubMed for query: (emotion recognition large language model cognitive appraisal empathy) NOT (psychosis NOT neuro* NOT Neuro* NOT EEG)

📄 arXiv 검색 시작
🔍 arXiv 검색 중: (cat:cs.CL OR cat:q-bio.NC) AND (emotion recognition large language model cognitive appraisal empathy)
📂 카테고리: cs.CL, q-bio.NC
✅ 검색된 논문 수: 25
💾 CSV 파일 저장 완료: literature_search_arxiv_20251102_235312.csv

Agent: Literature Search Agent (PubMed + arXiv)
[PubMed] Error: No papers found in PubMed for query: (emotion recognition large language model cognitive appraisal empathy) NOT (psychosis NOT neuro* NOT Neuro* NOT EEG)

[arXiv] Successfully found and saved 25 papers to 'literature_search_arxiv_20251102_235312.csv'


## 4. Agent 2: Introduction Writer Agent ✍️

초록을 기반으로 서론을 작성하는 에이전트입니다.

In [75]:
def analyze_abstract_structure(abstract: str) -> str:
    """
    Analyze the structure and key components of an abstract.
    
    Args:
        abstract: The abstract text to analyze
    
    Returns:
        Analysis of the abstract structure and main components
    """
    # 주요 구성요소 탐지
    has_background = any(word in abstract.lower() for word in ['research', 'study', 'literature', 'previous', 'studies show'])
    has_purpose = any(word in abstract.lower() for word in ['purpose', 'aim', 'objective', 'examine', 'investigate', 'this study'])
    has_method = any(word in abstract.lower() for word in ['method', 'participant', 'design', 'measure', 'procedure'])
    has_results = any(word in abstract.lower() for word in ['result', 'found', 'showed', 'demonstrated', 'significant'])
    has_conclusion = any(word in abstract.lower() for word in ['suggest', 'conclude', 'implication', 'finding'])
    
    # 단어 수
    word_count = len(abstract.split())
    
    analysis = "Abstract Structure Analysis:\n"
    analysis += f"- Word count: {word_count}\n"
    analysis += f"- Background/Context: {'✓ Present' if has_background else '✗ Missing'}\n"
    analysis += f"- Research Purpose: {'✓ Present' if has_purpose else '✗ Missing'}\n"
    analysis += f"- Method: {'✓ Present' if has_method else '✗ Missing'}\n"
    analysis += f"- Results: {'✓ Present' if has_results else '✗ Missing'}\n"
    analysis += f"- Conclusion: {'✓ Present' if has_conclusion else '✗ Missing'}\n\n"
    
    # 키워드 추출 (간단한 방식)
    words = abstract.lower().split()
    # 불용어 제거
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'was', 'were', 'this', 'that'}
    content_words = [w.strip('.,!?;:()') for w in words if w not in stop_words and len(w) > 4]
    
    from collections import Counter
    word_freq = Counter(content_words)
    top_keywords = [word for word, count in word_freq.most_common(10)]
    
    analysis += f"Key terms identified: {', '.join(top_keywords[:5])}\n"
    
    return analysis

def generate_introduction_outline(abstract: str) -> str:
    """
    Generate an outline for the introduction based on the abstract.
    
    Args:
        abstract: The abstract text
    
    Returns:
        A suggested outline for the introduction
    """
    outline = "Suggested Introduction Outline:\n\n"
    outline += "I. Opening: Broad Context and Importance\n"
    outline += "   - Start with the broader topic/issue\n"
    outline += "   - Explain why this research area is important\n\n"
    
    outline += "II. Literature Review: What We Know\n"
    outline += "   - Review existing research on the topic\n"
    outline += "   - Highlight key findings from previous studies\n"
    outline += "   - Identify patterns or trends in the literature\n\n"
    
    outline += "III. Research Gap: What We Don't Know\n"
    outline += "   - Point out limitations or gaps in existing research\n"
    outline += "   - Explain what remains unclear or unexplored\n\n"
    
    outline += "IV. Current Study: What This Study Does\n"
    outline += "   - State the purpose/aim of the current study\n"
    outline += "   - Present research questions or hypotheses\n"
    outline += "   - Briefly mention the approach/method\n\n"
    
    outline += "V. Significance: Why This Study Matters\n"
    outline += "   - Explain the contribution to the field\n"
    outline += "   - Discuss potential implications\n\n"
    
    return outline

def write_introduction_draft(abstract: str, style: str = "formal") -> str:
    """
    Write a draft introduction based on the abstract.
    
    Args:
        abstract: The abstract text
        style: Writing style (formal, accessible, or concise)
    
    Returns:
        A draft introduction text
    """
    # 추상에서 주요 정보 추출
    sentences = [s.strip() for s in abstract.split('.') if s.strip()]
    
    introduction = "DRAFT INTRODUCTION\n"
    introduction += "=" * 50 + "\n\n"
    
    # Opening paragraph (general context)
    introduction += "[PARAGRAPH 1: Opening - Broad Context]\n"
    introduction += "In recent years, there has been growing interest in... [Expand on the broader context of your research topic]. "
    introduction += "This area of research is important because... [Explain significance].\n\n"
    
    # Literature review
    introduction += "[PARAGRAPH 2-3: Literature Review]\n"
    introduction += "Previous research has demonstrated... [Cite relevant studies]. "
    introduction += "For example, studies have shown that... [Summarize key findings]. "
    introduction += "Additionally, other researchers have found... [More literature].\n\n"
    
    # Research gap
    introduction += "[PARAGRAPH 4: Research Gap]\n"
    introduction += "However, despite these advances, several questions remain unanswered. "
    introduction += "Specifically, it is still unclear... [Identify gap]. "
    introduction += "This gap in knowledge limits our understanding of... [Explain limitation].\n\n"
    
    # Current study
    introduction += "[PARAGRAPH 5: Current Study]\n"
    if sentences:
        # 초록의 첫 문장을 활용
        introduction += f"The present study addresses this gap by... [Based on: {sentences[0]}]. "
    else:
        introduction += "The present study addresses this gap by... "
    introduction += "We hypothesize that... [State hypotheses]. "
    introduction += "To test this, we... [Brief method].\n\n"
    
    # Significance
    introduction += "[PARAGRAPH 6: Significance]\n"
    introduction += "This research contributes to the field by... [Explain contribution]. "
    introduction += "The findings have important implications for... [Discuss implications].\n\n"
    
    introduction += "=" * 50 + "\n"
    introduction += "Note: This is a template draft. Please:\n"
    introduction += "1. Fill in the bracketed sections with specific content\n"
    introduction += "2. Add citations from your literature search\n"
    introduction += "3. Expand each paragraph with supporting details\n"
    introduction += "4. Ensure smooth transitions between paragraphs\n"
    
    return introduction

print("✅ Introduction Writer Tools 정의 완료")

✅ Introduction Writer Tools 정의 완료


In [79]:
def save_result_to_txt(result: dict, filename: str = None) -> str:
    """
    Save agent result to a txt file.
    
    Args:
        result: Agent result dictionary with 'agent' and 'result' keys
        filename: Output filename (auto-generated if None)
    
    Returns:
        Path to saved file
    """
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"agent_output_{timestamp}.txt"
    
    content = f"Agent: {result['agent']}\n"
    content += "="*60 + "\n\n"
    content += result['result']
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"💾 결과가 '{filename}'에 저장되었습니다.")
    return filename

def generate_introduction_from_txt(txt_file: str, style: str = "formal") -> str:
    """
    Read a txt file and generate an introduction draft using Google API.
    
    Args:
        txt_file: Path to the txt file containing abstract or initial draft
        style: Writing style (formal, accessible, or concise)
    
    Returns:
        Generated introduction with APA 7 references
    """
    # Read the txt file
    with open(txt_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # Create prompt for Google API
    prompt = f"""
You are an expert academic writer specializing in psychology research papers.

Task: Based on the following content, write a comprehensive introduction section for a research paper.

Content from file:
{content}

Writing style: {style}

IMPORTANT REQUIREMENTS:
1. Write a well-structured introduction following the standard academic format:
   - Opening with broad context
   - Literature review with relevant citations
   - Research gap identification
   - Current study purpose and hypotheses
   - Significance and implications

2. Follow the structure and outline provided in the txt file content above. Use it as a guide for organizing your introduction.

3. Include in-text citations in APA 7th edition format (Author, Year) throughout the introduction. Do NOT use bold text, asterisks, or any markdown formatting.

4. At the end of the introduction, provide a References section with all cited works in APA 7th edition format.

5. The introduction should be approximately 2,000 words in length.

6. Use plain text only - no bold, italics, or special formatting characters (avoid **, __, etc.).

7. Ensure the introduction flows logically and sets up the research effectively.

Please generate the introduction draft now with proper APA 7 citations and references.
"""
    
    # Use Google Gemini API
    model = genai.GenerativeModel(model_name=MODEL)
    response = model.generate_content(prompt)
    
    # Save the generated introduction
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_filename = f"introduction_draft_{timestamp}.txt"
    
    with open(output_filename, 'w', encoding='utf-8') as f:
        f.write("="*60 + "\n")
        f.write("INTRODUCTION DRAFT (Generated with APA 7 References)\n")
        f.write("="*60 + "\n\n")
        f.write(response.text)
    
    print(f"✅ Introduction draft가 '{output_filename}'에 저장되었습니다.")
    return output_filename

print("✅ File I/O and Introduction Generation Tools 정의 완료")

✅ File I/O and Introduction Generation Tools 정의 완료


In [ ]:
def introduction_writer_agent(abstract: str, style: str = "formal") -> dict:
    """
    Agent specialized in writing introduction sections from abstracts.
    
    Args:
        abstract: The abstract of the research paper
        style: Writing style (formal, accessible, or concise)
    """
    prompt = f"""
You are an expert academic writer specializing in psychology research papers.

Task: Write a comprehensive introduction section based on the following abstract:

{abstract}

Writing style: {style}

Steps:
1. Use analyze_abstract_structure() to understand the abstract components
2. Use generate_introduction_outline() to create a structured outline
3. Use write_introduction_draft() to generate a draft introduction
4. Provide guidance on what sections need more development

The introduction should follow the standard structure:
- Opening with broad context
- Literature review
- Research gap identification
- Current study purpose
- Significance and implications

Ensure the introduction is well-organized, flows logically, and sets up the research effectively.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[analyze_abstract_structure, generate_introduction_outline, write_introduction_draft],
        max_turns=5
    )
    
    return {
        "agent": "Introduction Writer Agent",
        "result": response.choices[0].message.content
    }

print("✅ Introduction Writer Agent 정의 완료")

In [80]:
# 예시 초록
sample_abstract = """
Large language models (LLMs) are advancing in their ability to identify cognitive appraisal and its corresponding emotional outcome in human-written texts. Recent findings show that LLM-based chatbots generate empathetic messages by reasoning about users’ cognitive appraisals and offer targeted reappraisals. However, it is unclear which appraisal dimensions they prioritize. In this work, we prompted the model to (1) identify the author’s emotion out of seven emotion categories (anger, disgust, fear, guilt, joy, sadness, and shame), (2) explain why that category was selected, and (3) rate appraisal intensity from human text (n=1366). We then measured the accuracy of emotion alignment (i.e., whether the models accurately infer the author's emotion) and label agreement of appraisal dimensions between human annotators from the dataset and the models. We found that GPT4o, GPT-o1, and Gemini-1.5-flash models showed the highest accuracy in identifying joy (average F1 score = .97) and the lowest accuracy in identifying shame (average F1 score = .46), often confusing it with guilt. There was only high agreement (k > .68) in pleasantness and responsibility, whereas the other appraisals showed low agreement (k < .5). We further discuss the implications of LLMs’ ability to understand humans’ emotions for enhanced human-AI collaboration and suggest future directions for refining their ability for emotional reasoning.
"""

print("✍️ Introduction Writer Agent 테스트\n")
print("="*60)
print("단계 1: 초록 분석 및 결과 저장\n")

# Agent 실행
result = introduction_writer_agent(
    abstract=sample_abstract,
    style="formal"
)

print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

# 결과를 txt 파일로 저장
print("\n" + "="*60)
saved_file = save_result_to_txt(result, "introduction_writer_analysis.txt")

✍️ Introduction Writer Agent 테스트

단계 1: 초록 분석 및 결과 저장

Agent: Introduction Writer Agent
The draft introduction provides a solid structural foundation, but several sections require significant development to become a comprehensive and well-supported academic introduction.

Here is guidance on what sections need more development:

1.  **Opening with Broad Context (Paragraph 1):**
    *   **Need for Development:** The current draft placeholder, "In recent years, there has been growing interest in... [Expand on the broader context of your research topic]. This area of research is important because... [Explain significance]," is very general.
    *   **Guidance:** Begin by establishing the profound impact and rapid advancements of Large Language Models (LLMs) in various domains, particularly their emerging capabilities in understanding and processing human language. Connect this to the significance of emotional intelligence and cognitive appraisal within human communication, setting the sta

In [81]:
# 단계 2: txt 파일을 읽어서 Google API로 완전한 introduction 생성
print("\n" + "="*60)
print("단계 2: Google API로 Introduction Draft 생성 (APA 7 References 포함)\n")

introduction_file = generate_introduction_from_txt(
    txt_file=saved_file,
    style="formal"
)

print("\n✅ 완료! 다음 파일들이 생성되었습니다:")
print(f"  1. {saved_file} - 초록 분석 결과")
print(f"  2. {introduction_file} - APA 7 references를 포함한 완전한 introduction draft")


단계 2: Google API로 Introduction Draft 생성 (APA 7 References 포함)

✅ Introduction draft가 'introduction_draft_20251103_001826.txt'에 저장되었습니다.

✅ 완료! 다음 파일들이 생성되었습니다:
  1. introduction_writer_analysis.txt - 초록 분석 결과
  2. introduction_draft_20251103_001826.txt - APA 7 references를 포함한 완전한 introduction draft


## 5. Agent 3: APA 7 Citation Checker Agent 📝
APA 7판 형식을 확인하는 에이전트입니다.

In [87]:
def extract_apa_guidelines(pdf_path: str) -> str:
    """
    Extract APA 7 citation guidelines from the PDF file.
    
    Args:
        pdf_path: Path to the APA 7 Style PDF file
    
    Returns:
        Extracted text content from the PDF
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ""
            # 처음 20페이지만 추출 (주요 citation 규칙이 있는 부분)
            for page_num in range(min(20, len(pdf_reader.pages))):
                text += pdf_reader.pages[page_num].extract_text()
        
        return f"APA 7 Guidelines (excerpt):\n{text[:3000]}..."
    except Exception as e:
        return f"Error reading PDF: {str(e)}. Using general APA 7 knowledge instead."

def check_citations_in_text(text: str) -> str:
    """
    Check for in-text citations in the provided text.
    
    Args:
        text: The manuscript text to check
    
    Returns:
        List of citations found in the text
    """
    # APA 형식 인용 패턴: (Author, Year) 또는 (Author et al., Year)
    citation_pattern = r'\([A-Z][a-z]+(?:\s+(?:et al\.|&|and)\s+[A-Z][a-z]+)?,\s*\d{4}[a-z]?\)'
    citations = re.findall(citation_pattern, text)
    
    if citations:
        unique_citations = list(set(citations))
        result = f"Found {len(unique_citations)} unique citations:\n"
        for i, cite in enumerate(unique_citations, 1):
            result += f"{i}. {cite}\n"
        return result
    else:
        return "No citations found in the text."

def check_reference_format(references: str) -> str:
    """
    Check if reference entries follow APA 7 format.
    
    Args:
        references: The reference list to check
    
    Returns:
        Analysis of reference formatting issues
    """
    issues = []
    lines = references.split('\n')
    
    for i, line in enumerate(lines, 1):
        if line.strip():
            # 기본 체크: 저자 이름이 있는지
            if not re.search(r'[A-Z][a-z]+,\s*[A-Z]', line):
                issues.append(f"Line {i}: Missing proper author format (Last, F. I.)")
            
            # 연도가 괄호 안에 있는지
            if not re.search(r'\(\d{4}[a-z]?\)', line):
                issues.append(f"Line {i}: Missing or incorrect year format (YYYY)")
    
    if issues:
        return "Reference format issues found:\n" + "\n".join(issues[:10])  # 최대 10개만 표시
    else:
        return "Reference format appears correct."

def check_citation_reference_match(text: str, references: str) -> str:
    """
    Check if all in-text citations have corresponding references.
    
    Args:
        text: The manuscript text
        references: The reference list
    
    Returns:
        Report on citation-reference matching
    """
    # 본문의 인용 추출
    citation_pattern = r'\(([A-Z][a-z]+(?:\s+(?:et al\.|&|and)\s+[A-Z][a-z]+)?),\s*(\d{4}[a-z]?)\)'
    citations = re.findall(citation_pattern, text)
    
    missing_refs = []
    for author, year in citations:
        # 참고문헌에 해당 저자와 연도가 있는지 확인
        if author not in references or year not in references:
            missing_refs.append(f"{author}, {year}")
    
    if missing_refs:
        unique_missing = list(set(missing_refs))
        return f"⚠️ Missing references for {len(unique_missing)} citations:\n" + "\n".join(unique_missing[:10])
    else:
        return "✓ All citations have corresponding references."

print("✅ APA Citation Checker Tools 정의 완료")

✅ APA Citation Checker Tools 정의 완료


In [88]:
def apa_citation_checker_agent(manuscript_text: str, references: str, pdf_path: str = "HW6/APA7-Style.pdf") -> dict:
    """
    Agent specialized in checking APA 7 citation format.
    """
    prompt = f"""
You are an APA 7th edition style expert.

Task: Check the following manuscript for APA 7 compliance:

Manuscript Text:
{manuscript_text[:2000]}...

References:
{references}

Steps:
1. Use check_citations_in_text() to find all citations
2. Use check_reference_format() to verify reference formatting
3. Use check_citation_reference_match() to ensure all citations have references
4. Provide a detailed report with specific issues and corrections needed

Focus on accuracy and provide actionable feedback.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[check_citations_in_text, check_reference_format, check_citation_reference_match],
        max_turns=5
    )
    
    return {
        "agent": "APA 7 Citation Checker Agent",
        "result": response.choices[0].message.content
    }

print("✅ APA Citation Checker Agent 정의 완료")

✅ APA Citation Checker Agent 정의 완료


### 테스트: APA Citation Checker Agent

In [90]:
# Introduction 파일 읽기 및 분리
import os

# introduction_draft 파일 경로 지정 (파일명을 직접 입력하세요)
introduction_file = "introduction_draft_20251103_001826.txt"  # 실제 파일명으로 변경하세요

# 파일이 존재하는지 확인
if os.path.exists(introduction_file):
    print(f"📄 읽을 파일: {introduction_file}\n")
    
    # 파일 읽기
    with open(introduction_file, 'r', encoding='utf-8') as f:
        content = f.read()
    
    # "References" 또는 "REFERENCES" 기준으로 분리
    if 'References' in content:
        split_point = content.find('References')
    elif 'REFERENCES' in content:
        split_point = content.find('REFERENCES')
    elif 'Reference' in content:
        split_point = content.find('Reference')
    else:
        # References 섹션이 없으면 전체를 본문으로
        split_point = len(content)
        print("⚠️ References 섹션을 찾을 수 없습니다. 전체를 본문으로 처리합니다.\n")
    
    # 본문과 참고문헌 분리
    sample_text = content[:split_point].strip()
    sample_references = content[split_point:].strip() if split_point < len(content) else ""
    
    print(f"✅ 본문 길이: {len(sample_text)} 자")
    print(f"✅ 참고문헌 길이: {len(sample_references)} 자\n")
    
else:
    print(f"⚠️ 파일을 찾을 수 없습니다: {introduction_file}")
    print("⚠️ 예시 데이터를 사용합니다.\n")
    
    # 예시 데이터
    sample_text = """
Recent studies have shown that cognitive training can improve working memory in older adults 
(Smith et al., 2023). This finding is consistent with previous research (Johnson & Lee, 2022).
However, some studies have found no significant effects (Brown, 2021).
"""
    
    sample_references = """
Brown, A. (2021). Cognitive training effects in aging. Journal of Gerontology, 45(3), 234-245.
Johnson, M., & Lee, S. (2022). Working memory interventions. Psychological Science, 33(2), 112-125.
"""

print("📝 APA Citation Checker Agent 테스트\n")
print("="*60)

result = apa_citation_checker_agent(sample_text, sample_references)

print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

📄 읽을 파일: introduction_draft_20251103_001826.txt

✅ 본문 길이: 15499 자
✅ 참고문헌 길이: 3481 자

📝 APA Citation Checker Agent 테스트

Agent: APA 7 Citation Checker Agent
Here is a detailed report on the APA 7th edition compliance of your manuscript:

**APA 7th Edition Compliance Report**

**1. General Manuscript Formatting**

*   **Issue:** The heading "INTRODUCTION DRAFT (Generated with APA 7 ref)" is not compliant with APA 7th edition guidelines.
*   **Correction:** The first heading in a manuscript (usually "Introduction") should be centered, bolded, and title case. Any additional descriptive text like "(Generated with APA 7 ref)" should be removed.
    *   **Example Correction:** **Introduction**

**2. In-Text Citations**

*   **Status:** All in-text citations appear to have a corresponding entry in your reference list.
*   **Details:** The following citations were identified in your text:
    *   (Brown et al., 2020)
    *   (OpenAI, 2023)
    *   (Picard, 1997)
    *   (Salovey & Mayer, 1990)
 

## 6. Agent 4: Coherence Checker Agent 🔗

문맥의 논리적 흐름과 일관성을 확인하는 에이전트입니다.

In [91]:
def check_cohesion(text: str) -> str:
    """
    Check the cohesion of the text using transition words and logical connectors.
    
    Args:
        text: The text to analyze
    
    Returns:
        Analysis of cohesion quality
    """
    # 연결어/전환어 목록
    transition_words = [
        'however', 'therefore', 'moreover', 'furthermore', 'nevertheless',
        'consequently', 'additionally', 'similarly', 'in contrast', 'for example',
        'specifically', 'particularly', 'notably', 'indeed', 'thus'
    ]
    
    text_lower = text.lower()
    found_transitions = [word for word in transition_words if word in text_lower]
    
    # 문장 수 계산
    sentences = text.split('.')
    sentence_count = len([s for s in sentences if s.strip()])
    
    transition_ratio = len(found_transitions) / max(sentence_count, 1)
    
    analysis = f"Cohesion Analysis:\n"
    analysis += f"- Sentences: {sentence_count}\n"
    analysis += f"- Transition words found: {len(found_transitions)}\n"
    analysis += f"- Transition ratio: {transition_ratio:.2f}\n"
    analysis += f"- Transitions used: {', '.join(found_transitions[:10])}\n\n"
    
    if transition_ratio < 0.1:
        analysis += "⚠️ Low cohesion: Consider adding more transition words to improve flow."
    elif transition_ratio > 0.3:
        analysis += "⚠️ High cohesion: May be using too many transition words."
    else:
        analysis += "✓ Good cohesion: Appropriate use of transition words."
    
    return analysis

def check_comprehension(text: str) -> str:
    """
    Check the comprehension level using readability metrics.
    
    Args:
        text: The text to analyze
    
    Returns:
        Readability analysis
    """
    # 간단한 가독성 지표 계산
    words = text.split()
    sentences = [s for s in text.split('.') if s.strip()]
    
    if not words or not sentences:
        return "Not enough text to analyze."
    
    avg_word_length = sum(len(word) for word in words) / len(words)
    avg_sentence_length = len(words) / len(sentences)
    
    # 긴 단어 비율 (7자 이상)
    long_words = [w for w in words if len(w) > 7]
    long_word_ratio = len(long_words) / len(words)
    
    analysis = f"Comprehension Analysis:\n"
    analysis += f"- Total words: {len(words)}\n"
    analysis += f"- Avg word length: {avg_word_length:.1f} characters\n"
    analysis += f"- Avg sentence length: {avg_sentence_length:.1f} words\n"
    analysis += f"- Long words (>7 chars): {len(long_words)} ({long_word_ratio:.1%})\n\n"
    
    # 가독성 판단
    if avg_sentence_length > 25:
        analysis += "⚠️ Sentences are too long. Consider breaking them down.\n"
    if long_word_ratio > 0.3:
        analysis += "⚠️ High proportion of complex words. Simplify vocabulary if possible.\n"
    if avg_sentence_length < 25 and long_word_ratio < 0.3:
        analysis += "✓ Good readability level.\n"
    
    return analysis

def check_vocabulary(text: str) -> str:
    """
    Check for appropriate vocabulary usage and repetition.
    
    Args:
        text: The text to analyze
    
    Returns:
        Vocabulary analysis
    """
    words = [w.lower().strip('.,!?;:') for w in text.split()]
    
    # 단어 빈도 계산
    from collections import Counter
    word_freq = Counter(words)
    
    # 불용어 제거
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'is', 'was', 'are', 'were'}
    content_words = {w: c for w, c in word_freq.items() if w not in stop_words and len(w) > 3}
    
    # 가장 자주 사용된 단어
    top_words = sorted(content_words.items(), key=lambda x: x[1], reverse=True)[:10]
    
    # 반복 확인 (3회 이상)
    repeated = [(w, c) for w, c in top_words if c >= 3]
    
    analysis = f"Vocabulary Analysis:\n"
    analysis += f"- Unique words: {len(set(words))}\n"
    analysis += f"- Lexical diversity: {len(set(words)) / len(words):.2f}\n"
    analysis += f"- Most frequent words: {', '.join([w for w, c in top_words[:5]])}\n\n"
    
    if repeated:
        analysis += f"⚠️ Frequently repeated words (consider synonyms):\n"
        for word, count in repeated[:5]:
            analysis += f"  - '{word}': {count} times\n"
    else:
        analysis += "✓ Good vocabulary variety.\n"
    
    return analysis

print("✅ Coherence Checker Tools 정의 완료")

✅ Coherence Checker Tools 정의 완료


In [92]:
def coherence_checker_agent(text: str) -> dict:
    """
    Agent specialized in checking text coherence and comprehension.
    """
    prompt = f"""
You are a writing quality expert specializing in academic prose.

Task: Analyze the following text for coherence, comprehension, and vocabulary:

{text}

Steps:
1. Use check_cohesion() to analyze logical flow
2. Use check_comprehension() to assess readability
3. Use check_vocabulary() to check word choice
4. Provide specific recommendations for improvement

Focus on helping improve the clarity and flow of academic writing.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[check_cohesion, check_comprehension, check_vocabulary],
        max_turns=5
    )
    
    return {
        "agent": "Coherence Checker Agent",
        "result": response.choices[0].message.content
    }

print("✅ Coherence Checker Agent 정의 완료")

✅ Coherence Checker Agent 정의 완료


### 테스트: Coherence Checker Agent

In [93]:
introduction_file = "introduction_draft_20251103_001826.txt"  # 실제 파일명으로 변경하세요

# 파일이 존재하는지 확인
if os.path.exists(introduction_file):
    print(f"📄 읽을 파일: {introduction_file}\n")
    
    # 파일 읽기
    with open(introduction_file, 'r', encoding='utf-8') as f:
        content = f.read()

sample_text = content

print("🔗 Coherence Checker Agent 테스트\n")
print("="*60)

result = coherence_checker_agent(sample_text)

print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

📄 읽을 파일: introduction_draft_20251103_001826.txt

🔗 Coherence Checker Agent 테스트

Agent: Coherence Checker Agent
Here's an analysis of your introduction draft, focusing on cohesion, comprehension, and vocabulary, along with recommendations for improvement:

**Cohesion Analysis:**

*   **Result:** The cohesion analysis indicates a low transition ratio (0.08) with only 8 transition words found across 100 sentences.
*   **Recommendation:** To improve the logical flow and connectedness of your ideas, consider incorporating more transition words and phrases. While you use some effective transitions like "however," "therefore," and "consequently," more explicit linking between sentences and paragraphs would enhance the reader's ability to follow your arguments. Examples of transitions that could be used more frequently include: "In addition," "Furthermore," "Moreover," "Conversely," "For instance," "As a result," etc. Also, ensure that topic sentences clearly signal the relationship of each pa

## 7. Agent 5: Summarizing Agent 📋

작성된 내용을 요약하는 에이전트입니다.

In [104]:
def extract_key_points(text: str) -> str:
    """
    Extract key points from the text.
    
    Args:
        text: The text to summarize
    
    Returns:
        List of key points
    """
    # 문장 분리
    sentences = [s.strip() for s in text.split('.') if s.strip()]
    
    # 중요 키워드
    important_keywords = [
        'significant', 'important', 'critical', 'essential', 'key', 'main',
        'demonstrated', 'showed', 'found', 'revealed', 'indicated',
        'however', 'therefore', 'thus', 'consequently'
    ]
    
    # 중요한 문장 점수 매기기
    scored_sentences = []
    for sent in sentences:
        score = sum(1 for kw in important_keywords if kw in sent.lower())
        scored_sentences.append((score, sent))
    
    # 점수 순으로 정렬
    scored_sentences.sort(reverse=True, key=lambda x: x[0])
    
    # 상위 5개 문장 선택
    key_points = [sent for score, sent in scored_sentences[:5] if score > 0]
    
    if key_points:
        result = "Key Points:\n"
        for i, point in enumerate(key_points, 1):
            result += f"{i}. {point}.\n"
        return result
    else:
        return "No key points identified with the current criteria."

def generate_abstract(text: str, max_words: int = 250) -> str:
    """
    Generate a brief abstract from the text.
    
    Args:
        text: The text to summarize
        max_words: Maximum words for the abstract
    
    Returns:
        A concise abstract
    """
    # Ensure max_words is an integer
    try:
        max_words = int(max_words)
    except (ValueError, TypeError):
        max_words = 250
    
    words = text.split()
    
    if len(words) <= max_words:
        return f"Abstract: {text}"
    
    # 처음 max_words개 단어 선택
    abstract_words = words[:max_words]
    abstract = ' '.join(abstract_words) + '...'
    
    return f"Abstract ({max_words} words): {abstract}"

print("✅ Summarizing Tools 정의 완료")

✅ Summarizing Tools 정의 완료


In [105]:
def summarizing_agent(text: str) -> dict:
    """
    Agent specialized in summarizing academic text.
    """
    # Limit text length to avoid token issues
    max_chars = 5000
    text_to_summarize = text[:max_chars] if len(text) > max_chars else text
    
    prompt = f"""
You are a summarization expert for academic papers.

Task: Summarize the following text and identify the main points:

{text_to_summarize}

Steps:
1. Use extract_key_points() to find the main arguments
2. Use generate_abstract() to create a concise summary
3. Verify that the summary captures the author's intended message
4. Suggest if any important points are missing or over-emphasized

Focus on accuracy and conciseness.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[extract_key_points, generate_abstract],
        max_turns=3
    )
    
    return {
        "agent": "Summarizing Agent",
        "result": response.choices[0].message.content
    }

print("✅ Summarizing Agent 정의 완료")

✅ Summarizing Agent 정의 완료


### 테스트: Summarizing Agent

In [106]:
introduction_file = "introduction_draft_20251103_001826.txt"  # 실제 파일명으로 변경하세요

# 파일이 존재하는지 확인
if os.path.exists(introduction_file):
    print(f"📄 읽을 파일: {introduction_file}\n")
    
    # 파일 읽기
    with open(introduction_file, 'r', encoding='utf-8') as f:
        content = f.read()

sample_text = content

print("📋 Summarizing Agent 테스트\n")
print("="*60)

result = summarizing_agent(sample_text)

print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

📄 읽을 파일: introduction_draft_20251103_001826.txt

📋 Summarizing Agent 테스트

Agent: Summarizing Agent
Large Language Models (LLMs) are transforming AI with their advanced language capabilities, but a key emerging area is the development of LLMs that can comprehend human emotions. This requires understanding emotional intelligence and, crucially, cognitive appraisal – the process by which individuals evaluate events and determine emotional responses. Key theories, such as Lazarus's cognitive-motivational-relational theory and Scherer's Component Process Model of Emotion, provide frameworks for AI to interpret human affect through primary and secondary appraisals and stimulus evaluation checks. This theoretical foundation is essential for moving beyond superficial sentiment analysis to a deeper, psychologically informed understanding of human emotion in AI systems, paving the way for truly emotionally intelligent and empathetic human-like interaction.

### Verification and Suggestions:

The

## 8. Agent 6: Figure Generation Agent 📊

연구 데이터를 시각화하는 에이전트입니다.

In [ ]:
def create_bar_chart(data_dict: str, title: str, xlabel: str, ylabel: str, filename: str = "figure_bar.png") -> str:
    """
    Create a bar chart from dictionary data.
    
    Args:
        data_dict: JSON string of data in format {"label1": value1, "label2": value2}
        title: Chart title
        xlabel: X-axis label
        ylabel: Y-axis label
        filename: Output filename
    
    Returns:
        Path to saved figure
    """
    try:
        data = json.loads(data_dict)
    except:
        # 예시 데이터 사용
        data = {"Treatment": 3.28, "Control": 0.38}
    
    plt.figure(figsize=(8, 6))
    plt.bar(data.keys(), data.values(), color=['#2196f3', '#ff9800'])
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    return f"Bar chart saved as {filename}"

def create_line_plot(x_values: str, y_values: str, title: str, xlabel: str, ylabel: str, filename: str = "figure_line.png") -> str:
    """
    Create a line plot from data.
    
    Args:
        x_values: JSON string of x-axis values
        y_values: JSON string of y-axis values
        title: Chart title
        xlabel: X-axis label
        ylabel: Y-axis label
        filename: Output filename
    
    Returns:
        Path to saved figure
    """
    try:
        x = json.loads(x_values)
        y = json.loads(y_values)
    except:
        # 예시 데이터
        x = [0, 2, 4, 6, 8]
        y = [20, 21, 23, 25, 28]
    
    plt.figure(figsize=(8, 6))
    plt.plot(x, y, marker='o', linewidth=2, markersize=8, color='#2196f3')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    return f"Line plot saved as {filename}"

def create_scatter_plot(x_values: str, y_values: str, title: str, xlabel: str, ylabel: str, filename: str = "figure_scatter.png") -> str:
    """
    Create a scatter plot from data.
    
    Args:
        x_values: JSON string of x-axis values
        y_values: JSON string of y-axis values
        title: Chart title
        xlabel: X-axis label
        ylabel: Y-axis label
        filename: Output filename
    
    Returns:
        Path to saved figure
    """
    try:
        x = json.loads(x_values)
        y = json.loads(y_values)
    except:
        # 예시 데이터
        x = np.random.randint(65, 85, 30)
        y = np.random.randint(20, 30, 30)
    
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, alpha=0.6, s=100, color='#2196f3')
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    return f"Scatter plot saved as {filename}"

print("✅ Figure Generation Tools 정의 완료")

In [ ]:
def figure_generation_agent(figure_type: str, description: str) -> dict:
    """
    Agent specialized in creating research figures.
    """
    prompt = f"""
You are a data visualization expert for academic publications.

Task: Create a {figure_type} with the following description:
{description}

Available tools:
- create_bar_chart: For comparing groups
- create_line_plot: For trends over time
- create_scatter_plot: For correlations

Choose the appropriate tool and create a publication-quality figure.
Use clear titles and labels that would be suitable for an academic paper.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[create_bar_chart, create_line_plot, create_scatter_plot],
        max_turns=3
    )
    
    return {
        "agent": "Figure Generation Agent",
        "result": response.choices[0].message.content
    }

print("✅ Figure Generation Agent 정의 완료")

### 테스트: Figure Generation Agent

In [ ]:
print("📊 Figure Generation Agent 테스트\n")
print("="*60)

result = figure_generation_agent(
    figure_type="bar chart",
    description="Compare mean MMSE score changes between treatment group (3.28) and control group (0.38)"
)

print(f"Agent: {result['agent']}")
print("="*60)
print(result['result'])

# 생성된 그림 표시
from IPython.display import Image, display
try:
    display(Image('figure_bar.png'))
except:
    print("(Figure file not found)")

## 9. Agent 7: Reader Accessibility Agent 👥

대학 신입생 수준에서 논문의 이해 가능성을 평가하는 에이전트입니다.

In [ ]:
def assess_readability_level(text: str) -> str:
    """
    Assess if the text is readable for undergraduate freshmen.
    
    Args:
        text: The text to evaluate
    
    Returns:
        Readability assessment
    """
    words = text.split()
    sentences = [s for s in text.split('.') if s.strip()]
    
    if not words or not sentences:
        return "Not enough text to analyze."
    
    # 기본 지표
    avg_word_length = sum(len(w) for w in words) / len(words)
    avg_sentence_length = len(words) / len(sentences)
    
    # 복잡한 단어 (8자 이상)
    complex_words = [w for w in words if len(w) >= 8]
    complex_ratio = len(complex_words) / len(words)
    
    # 전문 용어 (간단한 휴리스틱)
    technical_terms = ['cognition', 'neuroplasticity', 'hippocampus', 'prefrontal', 
                      'neurotransmitter', 'psychometric', 'phenomenological']
    found_technical = [t for t in technical_terms if t.lower() in text.lower()]
    
    assessment = "Readability Assessment for Undergraduate Freshmen:\n"
    assessment += f"- Average sentence length: {avg_sentence_length:.1f} words\n"
    assessment += f"- Complex words: {len(complex_words)} ({complex_ratio:.1%})\n"
    assessment += f"- Technical terms found: {len(found_technical)}\n\n"
    
    # 난이도 판정
    difficulty_score = 0
    
    if avg_sentence_length > 20:
        difficulty_score += 1
        assessment += "⚠️ Sentences are long and may be difficult for beginners.\n"
    
    if complex_ratio > 0.25:
        difficulty_score += 1
        assessment += "⚠️ High use of complex vocabulary.\n"
    
    if len(found_technical) > 3:
        difficulty_score += 1
        assessment += f"⚠️ Multiple technical terms used: {', '.join(found_technical[:5])}\n"
    
    if difficulty_score == 0:
        assessment += "\n✓ ACCESSIBLE: Suitable for undergraduate freshmen."
    elif difficulty_score == 1:
        assessment += "\n⚠️ MODERATE: Some simplification recommended."
    else:
        assessment += "\n❌ DIFFICULT: Significant simplification needed for freshmen."
    
    return assessment

def identify_jargon(text: str) -> str:
    """
    Identify technical jargon that may need explanation.
    
    Args:
        text: The text to analyze
    
    Returns:
        List of jargon terms
    """
    # 심리학 전문 용어 목록
    psychology_jargon = [
        'executive function', 'working memory', 'episodic memory', 'semantic memory',
        'cognitive load', 'neuroplasticity', 'prefrontal cortex', 'hippocampus',
        'meta-analysis', 'effect size', 'statistical power', 'p-value',
        'construct validity', 'internal validity', 'external validity',
        'phenomenology', 'psychometric', 'latent variable'
    ]
    
    text_lower = text.lower()
    found_jargon = [term for term in psychology_jargon if term.lower() in text_lower]
    
    if found_jargon:
        result = f"Technical jargon found ({len(found_jargon)} terms):\n"
        for term in found_jargon:
            result += f"  - {term}\n"
        result += "\n💡 Consider adding definitions or examples for these terms."
        return result
    else:
        return "No complex technical jargon detected."

def suggest_simplifications(text: str) -> str:
    """
    Suggest simplifications for complex sentences.
    
    Args:
        text: The text to analyze
    
    Returns:
        Simplification suggestions
    """
    sentences = [s.strip() for s in text.split('.') if s.strip()]
    
    long_sentences = []
    for i, sent in enumerate(sentences, 1):
        word_count = len(sent.split())
        if word_count > 25:
            long_sentences.append((i, sent, word_count))
    
    if long_sentences:
        result = "Suggestions for simplification:\n\n"
        for num, sent, count in long_sentences[:3]:  # 최대 3개만
            result += f"Sentence {num} ({count} words):\n"
            result += f'  "{sent[:100]}..."\n'
            result += "  💡 Consider breaking into shorter sentences.\n\n"
        return result
    else:
        return "Sentence length is appropriate. No major simplifications needed."

print("✅ Reader Accessibility Tools 정의 완료")

In [ ]:
def reader_accessibility_agent(text: str) -> dict:
    """
    Agent specialized in assessing accessibility for undergraduate freshmen.
    """
    prompt = f"""
You are an educational assessment expert specializing in psychology.

Task: Evaluate if the following text is accessible to undergraduate freshmen in psychology:

{text}

Steps:
1. Use assess_readability_level() to evaluate overall difficulty
2. Use identify_jargon() to find technical terms
3. Use suggest_simplifications() for improvement suggestions
4. Provide specific recommendations to make the text more accessible

Consider that freshmen may not be familiar with advanced statistical or theoretical concepts.
"""
    
    response = client.chat_completions_create(
        messages=[{"role": "user", "content": prompt}],
        tools=[assess_readability_level, identify_jargon, suggest_simplifications],
        max_turns=5
    )
    
    return {
        "agent": "Reader Accessibility Agent",
        "result": response.choices[0].message.content
    }

print("✅ Reader Accessibility Agent 정의 완료")

### 테스트: Reader Accessibility Agent

## 9. 통합 워크플로: 모든 Agent 함께 사용하기

이제 모든 에이전트를 순차적으로 실행하는 통합 워크플로를 만듭니다.

## 11. 결과 리포트 생성
모든 에이전트의 결과를 종합하여 최종 리포트를 생성합니다.

## 12. 요약 및 활용 가이드

### ✅ 구현된 7가지 Agent

| Agent | 기능 | 주요 도구 |
|-------|------|----------|
| 📚 Literature Search | PubMed 논문 검색 및 CSV 저장 | `search_pubmed()` (Biopython Entrez) |
| ✍️ Introduction Writer | 초록 기반 서론 작성 | `analyze_abstract_structure()`, `write_introduction_draft()` |
| 📝 APA Citation Checker | APA 7판 형식 검증 | `check_citations_in_text()`, `check_reference_format()` |
| 🔗 Coherence Checker | 문맥 및 논리적 흐름 분석 | `check_cohesion()`, `check_comprehension()` |
| 📋 Summarizing | 내용 요약 및 핵심 추출 | `extract_key_points()`, `generate_abstract()` |
| 📊 Figure Generation | 연구 그래프 생성 | `create_bar_chart()`, `create_line_plot()` |
| 👥 Reader Accessibility | 가독성 평가 (신입생 수준) | `assess_readability_level()`, `identify_jargon()` |

### 🎯 사용 시나리오

**1. 논문 작성 초기 단계**
- Literature Search Agent로 관련 논문 찾기 (CSV 저장)
  - Biopython Entrez 사용
  - MeSH 키워드 자동 추출
  - DOI 및 PMID 포함
- Introduction Writer Agent로 초록에서 서론 초안 작성

**2. 초안 작성 후**
- Coherence Checker로 논리적 흐름 확인
- Summarizing Agent로 작성 의도 확인

**3. 최종 검토 단계**
- APA Citation Checker로 형식 검증
- Reader Accessibility로 가독성 평가
- Figure Generation으로 시각자료 추가

### 💡 확장 아이디어

- **Translation Agent**: 한글-영문 번역
- **Plagiarism Checker**: 표절 검사
- **Statistical Reviewer**: 통계 분석 검증
- **Grammar Checker**: 문법 및 맞춤법 검사

### 📌 주요 개선 사항 (Literature Search Agent)

- **Biopython 사용**: `Bio.Entrez` 모듈로 전문적인 PubMed 검색
- **배치 처리**: 10개씩 묶어서 효율적으로 데이터 수집
- **풍부한 메타데이터**: PMID, DOI, MeSH 키워드 포함
- **진행 상황 표시**: 검색 및 수집 과정 실시간 출력
- **API 제한 준수**: 요청 간 0.5초 대기

### 전체 시스템 테스트

In [ ]:
# 예시 데이터
test_topic = "cognitive training effects in elderly adults"
test_keywords = "working memory, executive function, aging, neuroplasticity"

test_manuscript = """
Recent studies have demonstrated that cognitive training can improve working memory in older adults 
(Smith et al., 2023). This finding is consistent with previous research showing benefits of 
cognitive interventions (Johnson & Lee, 2022). The present study examined the effects of an 8-week 
cognitive training program on working memory capacity in adults aged 65-85. Participants in the 
treatment group showed significant improvements compared to controls. These results suggest that 
cognitive training can be an effective intervention for maintaining cognitive function in aging 
populations. However, future research should examine long-term effects.
"""

test_references = """
Johnson, M., & Lee, S. (2022). Working memory interventions in aging populations. Psychological Science, 33(2), 112-125.
Smith, A., Brown, B., & Davis, C. (2023). Cognitive training effects on executive function. Journal of Gerontology, 78(4), 456-470.
"""

# 전체 워크플로 실행
all_results = run_complete_paper_review(
    topic=test_topic,
    keywords=test_keywords,
    manuscript=test_manuscript,
    references=test_references
)

## 10. 결과 리포트 생성

모든 에이전트의 결과를 종합하여 최종 리포트를 생성합니다.

In [ ]:
def generate_final_report(results: dict, output_file: str = "paper_review_report.txt") -> str:
    """
    Generate a comprehensive report from all agent results.
    """
    report = """# 학술 논문 작성 Multi-Agent 시스템 종합 리포트

## 1. 문헌 검색 결과
{}

## 2. APA 7판 형식 검증
{}

## 3. 문맥 및 일관성 분석
{}

## 4. 내용 요약
{}

## 5. 그림 생성 결과
{}

## 6. 가독성 평가
{}

---
리포트 생성 일시: {}
""".format(
        results['literature']['result'],
        results['citation']['result'],
        results['coherence']['result'],
        results['summary']['result'],
        results['figure']['result'],
        results['accessibility']['result'],
        __import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    )
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(report)
    
    print(f"\n📄 최종 리포트가 '{output_file}'에 저장되었습니다.")
    return output_file

# 리포트 생성
report_file = generate_final_report(all_results)

# 파일 내용 출력
with open(report_file, 'r', encoding='utf-8') as f:
    print("\n" + "="*70)
    print("📋 최종 리포트 미리보기")
    print("="*70)
    print(f.read())

## 11. 요약 및 활용 가이드

### ✅ 구현된 6가지 Agent

| Agent | 기능 | 주요 도구 |
|-------|------|----------|
| 📚 Literature Search | Google Scholar 논문 검색 | `search_google_scholar()` |
| 📝 APA Citation Checker | APA 7판 형식 검증 | `check_citations_in_text()`, `check_reference_format()` |
| 🔗 Coherence Checker | 문맥 및 논리적 흐름 분석 | `check_cohesion()`, `check_comprehension()` |
| 📋 Summarizing | 내용 요약 및 핵심 추출 | `extract_key_points()`, `generate_abstract()` |
| 📊 Figure Generation | 연구 그래프 생성 | `create_bar_chart()`, `create_line_plot()` |
| 👥 Reader Accessibility | 가독성 평가 (신입생 수준) | `assess_readability_level()`, `identify_jargon()` |

### 🎯 사용 시나리오

**1. 논문 작성 초기 단계**
- Literature Search Agent로 관련 논문 찾기
- 인용할 논문 목록 작성

**2. 초안 작성 후**
- Coherence Checker로 논리적 흐름 확인
- Summarizing Agent로 작성 의도 확인

**3. 최종 검토 단계**
- APA Citation Checker로 형식 검증
- Reader Accessibility로 가독성 평가
- Figure Generation으로 시각자료 추가

### 💡 확장 아이디어

- **Translation Agent**: 한글-영문 번역
- **Plagiarism Checker**: 표절 검사
- **Statistical Reviewer**: 통계 분석 검증
- **Grammar Checker**: 문법 및 맞춤법 검사
